# Dilithium - Glitches - Predicted

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F4'
SS_VER = 'SS_VER_2_1'

fw_path = "../../../hardware/victims/firmware/simpleserial-dilithium-ref/simpleserial-dilithium-ref-{}.hex".format(PLATFORM)

TIMEOUT_SIGN_MS = 640
TIMEOUT_SIGN_NS = TIMEOUT_SIGN_MS * 1e6

# POLY_INDEX was previously called ITER_TARGET
POLY_INDEX = 60  # e.g.: POLY_INDEX = 0 means that a fault is issued after the loop where the index is 0; thus we have 4 non-zero coefficients and 252 zero coefficients

In [2]:
import logging
logging.basicConfig(level=logging.NOTSET)
logging.getLogger('io.github.alex1s.python-dilithium').setLevel(logging.WARNING)
logging.getLogger('gurobipy.gurobipy').setLevel(logging.WARNING) # please be quiet gurobi
logging.getLogger().setLevel(logging.DEBUG + 1) # default logger should not be used anyways!
__LOGGER = logging.getLogger('sigglitches')
__LOGGER.setLevel(logging.DEBUG)
logging.getLogger("usb_ctrl").setLevel(logging.WARNING)

In [3]:
import sys
if '../../../software' not in sys.path:
    sys.path.append('../../../software')
if 'python-dilithium' not in sys.path:
    sys.path.append('python-dilithium')
if 'dilithium_solver' not in sys.path:
    sys.path.append('dilithium_solver')

In [4]:
import chipwhisperer as cw
import importlib
import json
import uuid
import threading
import numpy as np
import ipywidgets as widgets
from chipwhisperer.capture.targets import TargetIOError, TargetTimeoutError
from dilithium import Dilithium
import struct
import random
from collections import defaultdict
import time
import math
from operator import itemgetter
import functools
import pickle

In [5]:
%%bash
make -C ../../../hardware/victims/firmware/simpleserial-dilithium-ref

make: Entering directory '/home/alexis/chipwhisperer.dilithium/hardware/victims/firmware/simpleserial-dilithium-ref'
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
make[1]: '.dep' is up to date.
SS_VER set to SS_VER_2_1
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc (15:9-2019-q4-0ubuntu1) 9.2.1 20191025 (release) [ARM/arm-9-branch revision 277599]
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling:
-en     simpleserial-dilithium-ref.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Compiling:
-en     .././simpleserial/simpleserial.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4_hal.c ...
mkdir -p "objdir-CW308_STM32F4"


In file included from .././hal/stm32f4/stm32f4_hal.c:3:
.././hal/stm32f4/stm32f4_hal_lowlevel.h:108: warning: "STM32F415xx" redefined
  108 | #define STM32F415xx
      | 
<command-line>: note: this is the location of the previous definition


-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4_hal_lowlevel.c ...
mkdir -p "objdir-CW308_STM32F4"


In file included from .././hal/stm32f4/stm32f4_hal_lowlevel.c:39:
.././hal/stm32f4/stm32f4_hal_lowlevel.h:108: warning: "STM32F415xx" redefined
  108 | #define STM32F415xx
      | 
<command-line>: note: this is the location of the previous definition


-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4_sysmem.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4xx_hal_rng.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Assembling: .././hal/stm32f4/stm32f4_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m4 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -fmessage-length=0 -ffunction-sections -DF_CPU=7372800 -Wa,-gstabs,-adhlns=objdir-CW308_STM32F4/stm32f4_startup.lst -I.././simpleserial/ -I.././hal -I.././hal/stm32f4 -I.././hal/stm32f4/CMSIS -I.././hal/stm32f4/CMSIS/core -I.././hal/stm32f4/CMSIS/device -I.././hal/stm32f4/Legacy -I.././crypto/ .././hal/stm32f4/stm32f4_startup.S -o objdir-CW308_STM32F4/stm32f4_startup.o
.
LINKING:
-en     simpleserial-dilithium-ref-CW308_STM32F4.elf ...
-e Done!
.
Creating load file for Flash: simpleserial-dilithium-ref-CW308_STM32F4.hex
arm-none-eabi-objcopy -O ihex -R .eeprom -R .fuse -R .lock -R .signature simpleserial-dilithium-ref-CW308_STM32F4.elf simples

In [6]:
try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope()

target = cw.target(scope, cw.targets.SimpleSerial2Dilithium)
#target.baud = 230400
target.scope = scope
target.con()

time.sleep(0.05)
target.dglitch_settings()  # dilithium glitch settings

d = target.dilithium

print("INFO: Found ChipWhisperer😍")

INFO: Found ChipWhisperer😍


In [7]:
# uncomment the following line to program the firmware; this takes a little while ...
# cw.program_target(scope, cw.programmers.STM32FProgrammer, fw_path)
target.reboot_flush()  # make sure the target is up and running for seamless future use

b'\xf3\xfb\x00\x0bb\x07boot ok\xc1\x00'

In [8]:
class Predictions:
    def __init__(self):
        self.__messages = []
        self.__signatures = []
        self.__signatures_no_fault = []
        
        def get_message_without_rejections(poly_index: int = None) -> (bytes, bytes):
            """Returns message, signature_packed"""
            if poly_index is None:
                poly_index = 1000  # this index is out of range thus it will not fault
            for i in range(2 ** 16 - 1):
                upper = i // 256
                lower = i % 256
                message = bytes([upper, lower])
                # __LOGGER.debug(f'Checking whether message {message}, faulted at polyvec_index "0" and poly_index "{poly_index}",  will sign without rejections ...')
                signature_packed, num_rejections = d.signature_faulted(message, target.secret_key, 0, poly_index)
                if num_rejections != 0:
                    message = None
                    continue
                else:
                    break
            if message is None:
                raise RuntimeException('We did not find a message without rejections searching two full bytes. While theoretically possible, it is more likely that the "signature_faulted" implementation is wrong.')
            return message, signature_packed
        
        for poly_index in range(0, d._polyz_unpack_num_iters):  # faulted at index _polyz_unpack_num_iters - 1 would mean no fault, because at that point in time all 256 coefficients already have been sampled
            message, signature = get_message_without_rejections(poly_index)
            self.__messages.append(message)
            self.__signatures.append(signature)
            self.__signatures_no_fault.append(d.signature(message, target.secret_key))
        assert len(set(self.__signatures)) == len(self.__signatures)
    
    def get_message_no_fault(self) -> bytes:
        return self.__messages[-1]
    
    def get_signature_no_fault(self) -> bytes:
        return self.__signatures[-1]

    def get_message_faulted(self, poly_index: int)  -> bytes:
        assert 0 <= poly_index < d._polyz_unpack_num_iters - 1
        return self.__messages[poly_index]
    
    def get_signature_faulted(self) -> bytes:
        assert 0 <= poly_index < d._polyz_unpack_num_iters - 1
        return self.__signatures[poly_index]
    
    def get_signature_faulted(self) -> bytes:
        assert 0 <= poly_index < d._polyz_unpack_num_iters - 1
        return self.__signatures[poly_index]
    
    def match(self, signature_packed: bytes) -> int:
        """
        Check if a signatures matches with a prediction.
        
        Returns -1 if it did not match with any prediction.
        Returns d._polyz_unpack_num_iters -1 if it matches the non-faulted prediction.
        Otherwise the return value i indicates that it matches the prediction of a fault
        after i + 1 iteration(s) / non-zero coefficient(s).
        """
        # first check if it is not faulted
        try:
            self.__signatures_no_fault.index(signature_packed)
            return d._polyz_unpack_num_iters - 1
        except ValueError:
            pass
        
        # see if it matches a expected fault pattern of any poly_index
        try:
            return self.__signatures.index(signature_packed)
        except ValueError:
            return -1
        
predictions = Predictions()
predictions

In [9]:
# these predictions are only needed when atting the loop
# poly_predictuons[i] = poly we would expect if faulted after iteration with _index_ i
target.loop()
poly_packed_no_fault = target.get_poly()
poly_no_fault = d._polyz_unpack(poly_packed_no_fault)
poly_predictions = []
for i in range(d._polyz_unpack_num_iters):
    split = (i + 1) * d._polyz_unpack_coeffs_per_iter
    poly_fault = np.concatenate((poly_no_fault[:split], np.zeros(d.n - split, dtype=np.int32)))
    assert np.shape(poly_fault) == (d.n,)
    poly_predictions.append(d._polyz_pack(poly_fault))

print(f"trig_count_loop = {target.loop_duration}; trig_count_loop_no_fault = {target.loop_duration_threshold}")
print(f'If we are running the extreme version per iteration the trigger is high for {target.loop_duration / 64} clock cycles')

trig_count_loop = 383; trig_count_loop_no_fault = 353
If we are running the extreme version per iteration the trigger is high for 5.984375 clock cycles


In [10]:
from enum import Enum
class AttackTarget(Enum):
    POLYZ_UNPACK = 'POLYZ_UNPACK'
    SIGNATURE = 'SIGNATURE'
attack_target = AttackTarget.POLYZ_UNPACK
attack_target.value

'POLYZ_UNPACK'

In [11]:
from typing import Union
def analyze_poly(subject: Union[np.ndarray, bytes], reference: Union[np.ndarray, bytes]) -> (Union[int, None], Union[int, None]):
    """
    Analyze the possibly faulted poly (subject) to a non-faulted (reference) one.
    
    Returns a pair of None if subject and reference do not differ.
    Otherwise the first integer of the pair is the number of leading coefficients which are the same
    and the second integer is the number of trailing zeros of the subject.
    """
    if type(subject) == bytes:
        subject = d._polyz_unpack(subject)
    if type(reference) == bytes:
        reference = d._polyz_unpack(reference)
    
    for i in range(d.n):
        if reference[i] != subject[i]:
            num_leading_same = i
            break
    else:  # all are the same
        return d.n, 0
    
    # at least one coefficient is different
    for i in range(num_leading_same, d.n):
        subject_trail = subject[i:]
        zeros =  np.zeros(np.shape(subject_trail))
        if np.all(subject_trail == zeros):
            return num_leading_same, np.shape(zeros)[0] 
        
    return num_leading_same, 0

In [12]:
def sign_estimate_zeros(z: np.ndarray) -> int:
    np.sum(np.abs(z) <= d.beta)

def sign_get_same_and_zero(z: np.ndarray) -> int:
    estimated_zeros = sign_estimate_zeros(z)
    estimated_same = d.n - estimated_zeros
    return estimated_same, estimated_zeros

def sign_action() -> None:
    pass
    #message_int = 0
    #try:
    #    target.sign(bytes([message_int], timeout=TIMEOUT_SIGN_MS)
        
    

In [13]:
# this cell defines all the functions and constants needed for one_try
get_index = lambda num_leading_same, num_trailing_zero: num_leading_same - num_leading_same % d._polyz_unpack_coeffs_per_iter
if attack_target == AttackTarget.POLYZ_UNPACK:
    action = target.loop
    get_result_packed = target.get_poly
    trig_count_threshold = target.loop_duration_threshold
    unpack_result = d._polyz_unpack
    get_same_and_zero = lambda poly_faulted: analyze_poly(poly_faulted, poly_no_fault)
    predictions = target.signature_predictions
    get_new_zeros = lambda poly: np.array((np.sum(poly == 0),) + (0,) * (d.l - 1))
    normal_trig_count = target.loop_duration
else:
    action = functools.partial(target.sign, timeout=TIMEOUT_SIGN_MS)
    get_result_packed = target.get_sig
    trig_count_threshold = target.loop_duration_sign_threshold
    unpack_result = lambda sig_packed: d._unpack_sig(sig_packed)[1]  # return z
    get_same_and_zero = sign_get_same_and_zero
    predictions = target.signature_predictions
    get_new_zeros = lambda z: np.sum(np.abs(z) <= d.beta, axis=1)
    normal_trig_count = target.loop_duration_sign

In [14]:
# glitch_spots = [ITER_TARGET * 62 - 73]
# line above should be equivalent to ITER_TARGET * 62 + 51
# lets think about the previous line: offset of 51 can not be a proper offset to fault as an iteration takes 62 clokc cycles
# and the first one takes a few cycles longer as it has to return from the trigger_high function and setup the loop
# thus we first valid offset has to be 51 + 61 = 112;
# the forumlar we need to use right now with POLY_INDEX is thus POLY_INDEX * 62 + 51
# with POLY_INDEX in [0, 1, ..., 62]
# width_start = 6.640625

# be aware that a iteration = poly_index + 1 or iteration - 1 = poly_index

# duration is 4699 for loop, but for sign it is way lower; it is: ????
# thus a iteration is at most 4699 / 64 = 74 for loop and ???? / 64 = ?? for sign

# following params caused 128 zeros:
#      ext_offset, offset, width
#      (2322, -3.515625, 0.78125) (2322, -2.34375, 2.734375)  (2322, 0.390625, 0.390625) (2322, 0.390625, 1.5625)
# (2322, 0.390625, 1.5625) has .727272 success rate ('num_leading_same': 124, 'num_trailing_zero': 128,)
# all meaningful results I found so far had a _very_ small offset and width, at most 15 but most of the times lower than 2

ext_offset_center = normal_trig_count // 2
single_loop_iteration_duration = math.ceil(target.loop_duration / d._polyz_unpack_num_iters)
half_single_loop_iteration_duration = math.ceil(target.loop_duration / d._polyz_unpack_num_iters / 2)
ext_offset_start = ext_offset_center - half_single_loop_iteration_duration
ext_offset_stop = ext_offset_center + half_single_loop_iteration_duration
print(f"A loop takes {target.loop_duration} clock cycles, thus we will hit a branch/compare instruction if we search through at least {2 * half_single_loop_iteration_duration} ext_offsets.")


offset_start = 0
offset_stop = 20

width_start = 0
width_stop = 20

if attack_target == AttackTarget.POLYZ_UNPACK:
    RES_FNAME = f'gc.results.pickled.shortcable-allinbuttight-{uuid.uuid4()}.json'
    
    ext_offset_start = 0
    ext_offset_stop = single_loop_iteration_duration + 1
    
    width_start = 0
    width_stop = 49.609375
    offset_start = -44.921875
    offset_stop = 49.609375

repeat = 1  # how often to glitch (in consecutive clock cycles)
redo = 10  # with a redo of e.g. 10 we find glitches with reliability >= 10%, right?

widths = target.widths_which_include(width_start, width_stop)
offsets = target.offsets_which_include(offset_start, offset_stop)
ext_offsets = list(range(ext_offset_start, ext_offset_stop + 1))

A loop takes 383 clock cycles, thus we will hit a branch/compare instruction if we search through at least 6 ext_offsets.


In [15]:
# uncomment following block if you want to search from the middle to the outside

#a =  ext_offsets[:len(ext_offsets) // 2][::-1]
#b =  ext_offsets[len(ext_offsets) // 2:]
#res = [None] * len(ext_offsets)
#res[1::2] = a
#res[::2] = b
#ext_offsets = res

ext_offsets

[0, 1, 2, 3, 4, 5, 6, 7]

In [16]:
offsets

[-44.921875,
 -44.53125,
 -44.140625,
 -43.75,
 -43.359375,
 -42.96875,
 -42.578125,
 -42.1875,
 -41.796875,
 -41.40625,
 -41.015625,
 -40.625,
 -40.234375,
 -39.84375,
 -39.453125,
 -39.0625,
 -38.671875,
 -38.28125,
 -37.890625,
 -37.5,
 -37.109375,
 -36.71875,
 -36.328125,
 -35.9375,
 -35.546875,
 -35.15625,
 -34.765625,
 -34.375,
 -33.984375,
 -33.59375,
 -33.203125,
 -32.8125,
 -32.421875,
 -32.03125,
 -31.640625,
 -31.25,
 -30.859375,
 -30.46875,
 -30.078125,
 -29.6875,
 -29.296875,
 -28.90625,
 -28.515625,
 -28.125,
 -27.734375,
 -27.34375,
 -26.953125,
 -26.5625,
 -26.171875,
 -25.78125,
 -25.390625,
 -25.0,
 -24.609375,
 -24.21875,
 -23.828125,
 -23.4375,
 -23.046875,
 -22.65625,
 -22.265625,
 -21.875,
 -21.484375,
 -21.09375,
 -20.703125,
 -20.3125,
 -19.921875,
 -19.53125,
 -19.140625,
 -18.75,
 -18.359375,
 -17.96875,
 -17.578125,
 -17.1875,
 -16.796875,
 -16.40625,
 -16.015625,
 -15.625,
 -15.234375,
 -14.84375,
 -14.453125,
 -14.0625,
 -13.671875,
 -13.28125,
 -12.890625,

In [17]:
widths

[0.390625,
 0.78125,
 1.171875,
 1.5625,
 1.953125,
 2.34375,
 2.734375,
 3.125,
 3.515625,
 3.90625,
 4.296875,
 4.6875,
 5.078125,
 5.46875,
 5.859375,
 6.25,
 6.640625,
 7.03125,
 7.421875,
 7.8125,
 8.203125,
 8.59375,
 8.984375,
 9.375,
 9.765625,
 10.15625,
 10.546875,
 10.9375,
 11.328125,
 11.71875,
 12.109375,
 12.5,
 12.890625,
 13.28125,
 13.671875,
 14.0625,
 14.453125,
 14.84375,
 15.234375,
 15.625,
 16.015625,
 16.40625,
 16.796875,
 17.1875,
 17.578125,
 17.96875,
 18.359375,
 18.75,
 19.140625,
 19.53125,
 19.921875,
 20.3125,
 20.703125,
 21.09375,
 21.484375,
 21.875,
 22.265625,
 22.65625,
 23.046875,
 23.4375,
 23.828125,
 24.21875,
 24.609375,
 25.0,
 25.390625,
 25.78125,
 26.171875,
 26.5625,
 26.953125,
 27.34375,
 27.734375,
 28.125,
 28.515625,
 28.90625,
 29.296875,
 29.6875,
 30.078125,
 30.46875,
 30.859375,
 31.25,
 31.640625,
 32.03125,
 32.421875,
 32.8125,
 33.203125,
 33.59375,
 33.984375,
 34.375,
 34.765625,
 35.15625,
 35.546875,
 35.9375,
 36.3281

In [18]:
normal_time = len(widths) * len(offsets) * len(ext_offsets) * (redo + 1)
print(normal_time)
t = TIMEOUT_SIGN_MS * normal_time / 1000
print(f'Expected sig time: {t}s = {t/60}min = {t/3600}h')
t_all = t * 1.36 # what was that exactly again?
print(f'Expected sig + transfer time: {t_all}s = {t_all/60}min = {t_all/3600}h')

2704592
Expected sig time: 1730938.88s = 28848.981333333333min = 480.8163555555555h
Expected sig + transfer time: 2354076.8768s = 39234.61461333334min = 653.9102435555557h


In [19]:
import chipwhisperer.common.results.glitch as glitch
gc = glitch.GlitchController(groups=["reset", "trig_cnt_high", "normal", "predicted", "zeros", "somefault"], parameters=["ext_offset", "offset", "width"])
gc.set_range("width", min(widths), max(widths))
gc.set_range("offset", min(offsets), max(offsets))
gc.set_range("ext_offset", min(ext_offsets), max(ext_offsets))
gc.display_stats()

IntText(value=0, description='reset count:', disabled=True)

IntText(value=0, description='trig_cnt_high count:', disabled=True)

IntText(value=0, description='normal count:', disabled=True)

IntText(value=0, description='predicted count:', disabled=True)

IntText(value=0, description='zeros count:', disabled=True)

IntText(value=0, description='somefault count:', disabled=True)

FloatSlider(value=0.0, continuous_update=False, description='ext_offset setting:', disabled=True, max=7.0, rea…

FloatSlider(value=-44.921875, continuous_update=False, description='offset setting:', disabled=True, max=49.60…

FloatSlider(value=0.390625, continuous_update=False, description='width setting:', disabled=True, max=49.60937…

In [20]:
# trig count is 3993 (most common) and 3994 (a bit less common) (ratio roughthly 2/4) when signing
# trig count is 4698 (most common) and 4699 (a bit less common) (ratio roughthly 864/2317) when looping

first_trig_count_counts = defaultdict(int)
first_trig_count_counts_widget = widgets.Textarea(
    value='DISABLED',
    placeholder='No data recorded yet ...',
    description='trig_count: count:',
    disabled=True   
)
def first_trig_count_counts_update(trig_count: int):
    return  # DISABLED
    first_trig_count_counts[trig_count] += 1
    value = ''
    for trig_count, count in sorted(first_trig_count_counts.items()):
        value += f'{trig_count}: {count}\n'
    first_trig_count_counts_widget.value = value
first_trig_count_counts_widget

Textarea(value='DISABLED', description='trig_count: count:', disabled=True, placeholder='No data recorded yet …

In [21]:
somefault_sigs = defaultdict(int)
somefault_widget = widgets.Textarea(
    placeholder='No somefault so far',
    description='String:',
    disabled=True
)
def somefault_update(sig: bytes) -> None:
    somefault_sigs[sig] += 1
    
    somefault_sigs_short = {}  
    for sig, count in somefault_sigs.items():
        # truncating like this should work as signature is so tightly packed that it is almost uniform random
        somefault_sigs_short[sig[:5]] = count
    
    somefault_widget.value = f'Count distinct = {len(somefault_sigs)}\n{somefault_sigs_short}'
somefault_widget

Textarea(value='', description='String:', disabled=True, placeholder='No somefault so far')

In [22]:
def get_stats_html() -> str:
    good_keys = ["predicted", "zeros"]
    good_groups = good_keys
    bad_groups = ["somefault", "reset", "trig_cnt_high", "normal"]
    groups_ordered = good_groups + bad_groups
    columns_ordered = gc.parameters + groups_ordered + ["total good", "total bad", "rate good (%)"] + ["trig_counts", "num_leading_same", "num_trailing_zero", "exact"]
    result_dict_good = {key: gc.results.result_dict[key] for key in gc.results.result_dict if key in good_keys}
    
#    parameters_visited = set()
#    for parameters in results_grouped:
#        if parameters[1:] not in parameters_visited:
#            res = get_linear_matches(parameters[1], parameters[2], good_groups)
#            if len(res) != 0:
#                print(parameters[1:])
#                print(res)
#                print()
#            parameters_visited.update([parameters[1:]])
    
        
    
    txt = ""
    txt += '<div id="dilithium-website">'
    txt += '<p style="font-family: Lucida Console, monospace">'
    
    txt += "<table>"
    td_start = f'<td style="padding: 3px;">'
    td_start_neutral = '<td style="padding: 3px; text-align: right">'
    td_start_good_group = f'<td style="padding: 3px; text-align: right; color: green;">'
    td_start_bad_group = f'<td style="padding: 3px; text-align: right; color: red;">'
    td_start_by_grop = {group: td_start_good_group if group in good_groups else td_start_bad_group for group in gc.groups}
    
    
    txt += f'<thead style="font-weight: bold"><tr>'
    for group in columns_ordered:
        txt += f"{td_start}{group}</td>"
    txt += f'</tr></thead>'
    

    results_grouped_items = list(results_grouped.items())
    results_grouped_items.sort(key=lambda x: (x[0][1:], x[0][0]))  # group by width and height, then sort these groups by ext_offset
    previous_offset_width = None
    for params, stats in results_grouped_items:
        if stats["zeros"] == 0:  # only take a look at those with zeros
            continue
        if previous_offset_width is not None and previous_offset_width != params[1:]:
            txt += '<tr style="border-top:1px solid black">'
        else:
            txt += "<tr>"
        previous_offset_width = params[1:]
        txt += td_start_neutral + '{}'.format(params[0]) + f'{type(params[0])}</td>'  # ext_offset without decimals
        for parameter in params[1:]:
            txt += td_start_neutral + '{:.6f}'.format(parameter) + f'{type(parameter)}</td>'
        total_good = 0
        total_bad = 0
        for group in groups_ordered:
            frequency = stats[group]
            if group in good_groups:
                total_good += frequency
            else:  # there is only good and bad
                total_bad += frequency
            txt += f'{td_start_by_grop[group]}{frequency}</td>'
        txt += f'{td_start_good_group}{total_good}</td>'
        txt += f'{td_start_bad_group}{total_bad}</td>'
        txt += f'{td_start_neutral}' + '{:.2f}'.format(total_good/(total_good + total_bad) * 100) + '</td>'
        
        # now the sets ...
        metadatas = list(filter(lambda x: x is not None, stats["metadata"]))
        
        trig_counts = set(map(lambda md: md["trig_count"], metadatas))
        num_leading_same = set(map(lambda md: md["num_leading_same"], metadatas))
        num_trailing_zero = set(map(lambda md: md["num_trailing_zero"], metadatas))
        txt += f'{td_start_neutral}{trig_counts}</td>'
        txt += f'{td_start_neutral}{num_leading_same}</td>'
        txt += f'{td_start_neutral}{num_trailing_zero}</td>'
        
        exact = all(map(lambda x: x["packed"] == metadatas[0]["packed"], metadatas[1:]))
        txt += f'{td_start_neutral}{exact}</td>'
            
        
        txt += '</tr>'
    txt += "</table>"
    
#    for t, results in result_dict_good.items():
#        txt += '<div style="font-weight: bold;">' + t + ":\n</div>"
#        for result in results:
#            txt += "<div>\tparameters: " + str(result['parameters']) + "</div>\n"
#            txt += "\t<div>metadata: " + str({key: result['metadata'][key] for key in result['metadata'] if key != "packed" and key != "unpacked"}) + "</div>\n"
#            
#            if print_coeffs:
#                txt += '<table><tr>'
#                for i, coefficient in enumerate(result['metadata']["unpacked"]):
#                    if poly_no_fault[i] == coefficient:
#                        color = "green"
#                    else:
#                        color = "red"
#                    txt += f'<td style="padding: 3px; color: {color}; text-align: right;">{coefficient}</td>'
#                    if (i + 1) % 8 == 0:
#                        txt += "</tr><tr>"
#                txt += "</tr></table>"
#
#        txt += "\n"
#    txt += "</p>"    
#    txt += "</div>"  # dilithium-website

    return txt

In [23]:
#np.set_printoptions(linewidth=70)
result_widget = widgets.HTML(
    placeholder='Waiting for a first update ...',
    description='',
    disabled=True,
    layout=widgets.Layout(width='100%')
)

# enable again after tests!!!
results_grouped = defaultdict(lambda: {**{key: 0 for key in gc.groups}, **{"metadata": [], "strdesc": []}})

gc_lock = threading.Lock()
def gc_add(group, parameters, strdesc=None, metadata=None):
    with gc_lock:
        gc.add(group, parameters, strdesc, metadata)
    # results_grouped[parameters][group] += 1
    # if metadata is not None:
    #     results_grouped[parameters]["metadata"].append(metadata)
    # if strdesc is not None:
    #     results_grouped[parameters]["strdesc"].append(strdesc)
    # if group == "predicted" or group == "zeros" or group == "somefault" or results_grouped[parameters]["zeros"] != 0:
    #     update_result_widget()

        
def get_linear_matches(offset: float, width: float, good_groups: [str]) -> None:
    all_ext_offsets = set()
    for parameters, stats in results_grouped.items():
        if parameters[gc.parameters.index("offset")] != offset:
            continue
        if parameters[gc.parameters.index("width")] != width:
            continue
            
        for group in good_groups:
            if stats[group] != 0:
                all_ext_offsets.update([parameters[gc.parameters.index("ext_offset")]])
    return sorted(list(all_ext_offsets))

result_widget

HTML(value='', layout=Layout(width='100%'), placeholder='Waiting for a first update ...')

In [24]:
def update_result_widget(print_coeffs: bool = False):
    html = get_stats_html()
    result_widget.value = html
update_result_widget()

In [25]:
MIN_ZEROS = d.n *  2 # double to account for false positive classifications; may still be too little for late glitches
zeros = np.zeros(d.l)
zero_widgets = list((widgets.IntSlider(
    value=0,
    min=0,
    max=MIN_ZEROS + d.n, # most of the time we get a few more
    step=1,
    description=f"zeros[{i}]",
    disabled=True,
    continuous_update=True,
    orientation='horizontal',
    readout=True) for i in range(d.l)))
print(f'We need at least {MIN_ZEROS} zeros.')
display(*zero_widgets)

We need at least 512 zeros.


IntSlider(value=0, description='zeros[0]', disabled=True, max=768)

IntSlider(value=0, description='zeros[1]', disabled=True, max=768)

IntSlider(value=0, description='zeros[2]', disabled=True, max=768)

IntSlider(value=0, description='zeros[3]', disabled=True, max=768)

In [26]:
#target.run_without_glitch(functools.partial(target.sign, b"\x00"))
target.run_without_glitch(target.loop)
print(target.scope.adc.trig_count)

383


In [27]:
print(target.scope.adc.trig_count)

383


In [28]:
sigs_faulted = []
sigs_faulted_params = []

In [29]:
target.reboot_flush()

b'\x00\x0bb\x07boot ok\xc1\x00'

In [30]:
# just for attacking loops isolated
def one_try_new(param_tuple, recurse: bool = True) -> np.ndarray:
    """
    Try one fault parameter set.

    :param param_tuple: the parameters of the glitch (ext_offset, offset, width)
    :param action: what action is going to be glitched (polyz_unpack or signature)
    :param get_result_packed: callable 
    :returns: an upper bound on new zero coefficients
    """
    new_zeros = np.zeros(d.l)
    scope.sc.arm(False)  # reset trig_count
    scope.arm()
    try:
        action()
    except TargetIOError:  # corrupted or no response from the target; we could but do not really care to differentiate
        gc_add("reset", param_tuple)
        target.reboot_flush()
        return new_zeros
    
    trig_count = scope.adc.trig_count
    first_trig_count_counts_update(trig_count)
    metadata = {"trig_count": trig_count}
    
    if trig_count > trig_count_threshold:  # trigger was high too long; _very_ likely no loop abort
        gc_add("trig_cnt_high", param_tuple)
        return new_zeros
    
    # if you land here, _very_ likely it was some kind of loop abort
    
    packed = get_result_packed()
    metadata["packed"] = packed
    unpacked = unpack_result(packed)
    metadata["unpacked"] = unpacked
    num_leading_same, num_trailing_zero = get_same_and_zero(unpacked)
    metadata["num_leading_same"] = num_leading_same
    metadata["num_trailing_zero"] = num_trailing_zero

    new_zeros = get_new_zeros(unpacked)
    metadata["index"] = get_index(num_leading_same, num_trailing_zero)
    
    def get_stats(group: str, do_reset: bool = False) -> None:
        if not recurse:  # avoid infinite recursion
            return
        if results_grouped[param_tuple][group] != 1:  # only get stats once per parameter set
            return
        for _ in range(100):
            one_try_new(param_tuple, recurse=False)
    
    check_next_called = False
    def check_next() -> None:
        check_next_called = True
        if not recurse:
            return
        __LOGGER.info(f"Checking next for parameters {param_tuple} ...")
        for i in range(target.loop_duration + 1):
            new_ext_offset = i
            target.scope.glitch.ext_offset = new_ext_offset
            new_params = (new_ext_offset,) + param_tuple[1:]
            for _ in range(redo + 1):
                one_try_new(new_params, recurse=False)
        target.scope.glitch.ext_offset = param_tuple[0]  # restore ext_offset, just in case ...
        __LOGGER.info(f"Done! {param_tuple} ...")
    
    prediction = predictions.get(packed)
    metadata['prediction'] = prediction
    if prediction is not None and prediction == d._polyz_unpack_num_iters - 1:
        gc_add("normal", param_tuple, metadata=metadata)
    elif prediction is not None and prediction != d._polyz_unpack_num_iters - 1:
        gc_add("perfect", param_tuple, metadata=metadata)
        get_stats("perfect")
        if not check_next_called:
            check_next()
        check_next_called = True
    elif num_trailing_zero >= d._polyz_unpack_coeffs_per_iter:
        gc_add("zeros", param_tuple, metadata=metadata)
        target.reboot_flush()  # we do not know what happened, better reset
        get_stats("zeros", do_reset=True)
        if not check_next_called:
            check_next()
        check_next_called = True
    elif prediction == d._polyz_unpack_num_iters - 1:
        gc_add("normal", param_tuple, metadata=metadata)
    else:
        gc_add("somefault", param_tuple, metadata=metadata) # we do not know what happened, better reset
        target.reboot_flush()

    update_result_widget()
    return new_zeros

In [31]:
# just for attacking loops isolated
def one_try_loop(param_tuple, action: callable, get_result_packed: callable, recurse: bool = True) -> np.ndarray:
    """
    Try one fault parameter set.

    :param param_tuple: the parameters of the glitch (ext_offset, offset, width)
    :param action: what action is going to be glitched (polyz_unpack or signature)
    :param get_result_packed: callable 
    :returns: an upper bound on new zero coefficients
    """
    new_zeros = np.zeros(d.l)
    scope.sc.arm(False)  # reset trig_count
    scope.arm()
    try:
        action()
    except TargetIOError:  # corrupted or no response from the target; we could but do not really care to differentiate
        gc_add("reset", param_tuple)
        target.reboot_flush()
        return new_zeros
    
    trig_count = scope.adc.trig_count
    first_trig_count_counts_update(trig_count)
    metadata = {"trig_count": trig_count}
    
    if trig_count > target.loop_duration_threshold:  # trigger was high too long; _very_ likely no loop abort
        gc_add("trig_cnt_high", param_tuple)
        return new_zeros
    
    # if you land here, _very_ likely it was some kind of loop abort
    
    poly_packed = get_result_packed()
    metadata["packed"] = poly_packed
    poly = d._polyz_unpack(poly_packed)
    metadata["unpacked"] = poly
    num_leading_same, num_trailing_zero = analyze_poly(poly, poly_no_fault)
    metadata["num_leading_same"] = num_leading_same
    metadata["num_trailing_zero"] = num_trailing_zero

    new_zeros[0] = np.sum(poly == 0)
    
    hit_poly_index = None
    if num_leading_same + num_trailing_zero == d.n and num_leading_same % d._polyz_unpack_coeffs_per_iter == 0 and num_trailing_zero > 0:
        hit_poly_index == num_leading_same // d._polyz_unpack_coeffs_per_iter
    
    metadata["index"] = hit_poly_index
    
    def get_stats(group: str, do_reset: bool = False) -> None:
        if not recurse:  # avoid infinite recursion
            return
        if results_grouped[param_tuple][group] != 1:  # only get stats once per parameter set
            return
        for _ in range(100):
            one_try_loop(param_tuple, action, get_result_packed, recurse=False)
    
    check_next_called = False
    def check_next() -> None:
        check_next_called = True
        if not recurse:
            return
        __LOGGER.info(f"Checking next for parameters {param_tuple} ...")
        for i in range(target.loop_duration + 1):
            new_ext_offset = i
            target.scope.glitch.ext_offset = new_ext_offset
            new_params = (new_ext_offset,) + param_tuple[1:]
            for _ in range(redo + 1):
                one_try_loop(new_params, action, get_result_packed, recurse=False)
        target.scope.glitch.ext_offset = param_tuple[0]  # restore ext_offset, just in case ...
        __LOGGER.info(f"Done! {param_tuple} ...")
            
    if num_leading_same == d.n:
        gc_add("normal", param_tuple, metadata=metadata)
    elif hit_poly_index is not None:
        gc_add("perfect", param_tuple, metadata=metadata)
        get_stats("perfect")
        if not check_next_called:
            check_next()
        check_next_called = True
    elif num_trailing_zero > 0:
        gc_add("zeros", param_tuple, metadata=metadata)
        target.reboot_flush()  # we do not know what happened, better reset
        get_stats("zeros", do_reset=True)
        if not check_next_called:
            check_next()
        check_next_called = True
    else:
        gc_add("somefault", param_tuple, metadata=metadata) # we do not know what happened, better reset
        target.reboot_flush()
        
    update_result_widget()
    return new_zeros

In [32]:
def one_try(scope, target, gc, sigs_faulted: list, sigs_faulted_params: list) -> bool:
    new_zeros = np.zeros(d.l)
    param_tuple = (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset)
    message = predictions.get_message_faulted(ITER_TARGET)
    metadata = {}
    try:
        timings, counts, trig_counts, timeout = target.sign_no_rej(message, TIMEOUT_SIGN)
        metadata["measured_time"] = timings, counts, trig_counts
        first_trig_count_counts_update(trig_counts[0])
        if timeout is not None:  # timeout!
            __LOGGER.warning(f"timeout at: {timeout}")
            gc.add(timeout, param_tuple, metadata=metadata)
    except TargetTimeoutError as e:
        gc.add("reset", param_tuple)
        __LOGGER.info(f'failed due to read timeout: {e}')
        target.reboot_flush()
        return new_zeros
    except TargetIOError as e:
        gc.add("reset", param_tuple)
        __LOGGER.info(f'failed due to TargetIOError error: {e}')
        target.reboot_flush()
        return new_zeros
    else:
        try:
            sig_candidate_packed = target.get_sig()
            _, z_candidate, _ = d._unpack_sig(sig_candidate_packed)
            new_zeros = np.sum(np.abs(z_candidate) <= d.beta, axis=1)
            
            prediction = predictions.match(sig_candidate_packed)
            metadata["prediction"] = prediction
            
            if prediction == ITER_TARGET:
                gc.add("perfect", param_tuple, metadata=metadata)

                # non trivial to estimate (at least for me, right now)
                # gc.add(f"perfect_{(min_num_nonzero - (d.l - 1) * d.n) // d._polyz_unpack_coeffs_per_iter}", param_tuple)
                new_zeros = np.sum(np.abs(z_candidate) <= d.beta, axis=1)
                return new_zeros
            
            if prediction == d._polyz_unpack_num_iters - 1:
                gc.add("normal", param_tuple, metadata=metadata)
                return new_zeros
            
            if prediction >= 0:
                # it faulted, but it hit a different iteration
                gc.add("miss", param_tuple, strdesc="Targetet iteration '{ITER_TARGET}' but instead hit '{prediction}'.", metadata=metadata)
                return new_zeros
                        
            max_num_zero = np.sum(np.abs(z_candidate) <= d.beta)
            min_num_nonzero = np.sum(np.abs(z_candidate) > d.beta)
            min_num_zero_needed = d.n - (ITER_TARGET + 1) * d._polyz_unpack_coeffs_per_iter
            __LOGGER.info(f'We have at most faulted {max_num_zero} y coefficients to zero. (expected at least: {min_num_zero_needed})')
            if max_num_zero >= min_num_zero_needed:
                gc.add("success", param_tuple)

                sigs_faulted.append(sig_candidate_packed)
                sigs_faulted_params.append(param_tuple)
                                
                return new_zeros

            
            # otherwise it is some other fault I can not explain
            somefault_update(sig_candidate_packed)
            gc.add("somefault", param_tuple)
            
            return new_zeros
        except TargetIOError as e:
            gc.add("reset", param_tuple)
            __LOGGER.info(f'Failed to read sig: {e}')
            target.reboot_flush()
    return new_zeros

In [33]:
def save_results():
    with open(RES_FNAME, 'wb') as f:
        pickle.dump(gc.results, f)

In [34]:
def save_thread() -> None:
    while True:
        with gc_lock:
            save_results()
        time.sleep(2 * 60)  # two minutes
        
thread = threading.Thread(target=save_thread)

In [35]:
thread.start()
logging.getLogger("ChipWhisperer Target").setLevel(logging.WARNING + 1)  # disable WARNING messages like "Read timed out: " or "Read timed out: Wÿ+"

start_time = time.time()
do_break = False
for ext_offset in ext_offsets:
    for offset in offsets:
        for width in widths:
            for _ in range(redo):
                gc.widget_list_parameter[gc.parameters.index("ext_offset")].value = ext_offset
                gc.widget_list_parameter[gc.parameters.index("offset")].value = offset
                gc.widget_list_parameter[gc.parameters.index("width")].value = width

                scope.glitch.ext_offset = ext_offset
                scope.glitch.offset = offset
                scope.glitch.width = width
                scope.glitch.repeat = repeat
                
                param_tuple = ext_offset, offset, width

                # new_zeros = one_try(scope, target, gc, sigs_faulted, sigs_faulted_params)
                # new_zeros = one_try_loop(param_tuple, action=target.loop, get_result_packed=target.get_poly)
                new_zeros = one_try_new(param_tuple)
                zeros += new_zeros
                for i, zero_widget in enumerate(zero_widgets):
                    zero_widget.value = zeros[i]

                # if np.all(zeros > MIN_ZEROS):
                #     do_break = True

                if do_break:
                    break
end_time = time.time()
total_duration = end_time - start_time
print(f'total duration: {total_duration}s {total_duration/60}min {total_duration/3600}h')

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

In [36]:
save_results()

In [37]:
result_data = {
    "glitch_results": gc.results.__dict__,
#    "sig_correct": list(sig_correct),
#    "glitch_spots": list(glitch_spots),
    "width_start": width_start,
    "width_stop": width_stop,
    "offset_start": offset_start,
    "offset_stop": offset_stop,
    "repeat": repeat,
#    "redo": redo,
#    "step": step,
    "sigs_faulted": list(map(lambda s: list(s), sigs_faulted)),
    "sigs_faulted_param_descripton": "0 = ext_offset, 1 = offset, 2 = width",
    "sigs_faulted_params": sigs_faulted_params
}
with open(f'newnewnew_{attack_target.value}_{uuid.uuid4()}.json', 'w') as f:
    json.dump(result_data, f)
#json.dumps(result_data)

TypeError: Object of type bytes is not JSON serializable

In [ ]:
exithere

In [ ]:
sigs_faulted_unpacked = list(map(lambda sig_packed: d._unpack_sig_full(sig_packed), sigs_faulted))
from dilithium_solver.signature import Signature, calculate_c_matrix_np
from dilithium_solver.recover_s_1_entry import recover_s_1_entry
from dilithium_solver.parameters import Parameters

params = Parameters.get_nist_security_level(d.nist_security_level)

sigs = list(map(
    lambda sig_faulted: Signature(
        sig_faulted[1],
        sig_faulted[0],
        calculate_c_matrix_np(sig_faulted[0], params)
    ), sigs_faulted_unpacked))
s_1_entry_index = 0


s_1 = d._unpack_sk(sk)[4]
timeout = 10
threshold = d.beta
for i in range(len(sigs_faulted_unpacked[0][1])): # long version of saying "l"
    result = recover_s_1_entry(sigs, i, s_1, params, 142387, timeout, threshold) # this number is not relevant
    print(result)

In [ ]:
scope.dis()
target.dis()